# Estimate doctors benefit & risk score
* use static network structure for estimation
* use adjacency matrix and cotor info files for calculation (number of patients, maximal capacity, shared patients,..)
* B and R for every doctor of each speciality 

In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
from os.path import join

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
### simulation settings
min_pats = 2
max_distance = 100
ptype = 'total'
ctype = 'hour-based'
tf = 'quarterly'
network = 'Österreich'
threshold = 0.9

doc_list = ['IM','KI','PSY','ORTR','RAD','DER','URO','HNO','CH','NEU','AU','GGH','AM']
risk_benefit = pd.DataFrame()

In [ ]:
for doc in doc_list:
    doc_file = 'doctor_info_bez={}_spec={}_ptype={}_ctype={}_tf={}_th={}.csv'.format(network, doc, ptype,
                                                                                     ctype, tf, threshold)  
    doctor_info = pd.read_csv(join('data', doc_file), delimiter=',')
    doc_IDs = doctor_info['adj_index']

    ### load adjacency matrix
    adj = sparse.load_npz('data/adj_all_doctors.npz')
    adj = adj.todense()
    # load distance matrix between docs
    dist_docs = sparse.load_npz('data/DistanceMatrixDocs.npz')
    dist_docs = dist_docs.todense()

    ### set connections with less than min_pats to zero
    adj[adj<min_pats] = 0

    ### set connections further than max distance to zero
    adj[dist_docs>max_distance] = 0
    adj = adj[np.ix_(doc_IDs, doc_IDs)]

    ### set diagonal to zero
    np.fill_diagonal(adj,0)
    adj = np.asarray(adj)

    doctor_info['initial_free_capacity'] = doctor_info.capacity - doctor_info.number_of_patients

    doctor_info['connections'] = ''
    doctor_info['num_of_connections'] = 0
    for d in doctor_info.index.values:
        doctor_info.at[d,'connections'] = list(np.where(adj[d,:]>0)[0])
        doctor_info.at[d,'num_of_connections'] = len(list(np.where(adj[d,:]>0)[0]))

    ## Risk
    #* R_i = mean( min( (Nj + Ni * wj)/Cj, 1) )j

    adj = adj.astype(float,copy=False)

    for i in range(len(adj)):
        adj[:,i] = adj[:,i]/np.sum(adj[:,i])

    doctor_info['Risk'] = 0 
    for d in doctor_info.index.values:
        N_i = doctor_info.loc[d,'number_of_patients'].item()
        connect = np.asarray(doctor_info.loc[d,'connections'])
        l = []
        for c in connect:
            w_j = adj[c,d]
            N_j = doctor_info.loc[c,'number_of_patients'].item()
            C_j = doctor_info.loc[c,'capacity'].item()
            l.append(np.min([(N_j+N_i*w_j)/C_j,1]))
        if len(l)>0:
            doctor_info.loc[d,'Risk'] = np.mean(np.asarray(l))
        else:
            doctor_info.loc[d,'Risk'] = np.nan

    ## Benefit
    #* B_i = initial free capacity of i

    doctor_info['Benefit'] = 0 
    for d in doctor_info.index.values:
        doctor_info.loc[d,'Benefit'] = doctor_info.loc[d,'capacity'] - doctor_info.loc[d,'number_of_patients']
        
        
        
    ### normalize benefit score
    normalized_df=(doctor_info.Benefit-np.min(doctor_info.Benefit))/(np.max(doctor_info.Benefit)-np.min(doctor_info.Benefit))
    doctor_info.Benefit = normalized_df
    
    
    doctor_info = doctor_info.drop(columns=['connections','number_of_patients','capacity','gemeinde']).round(3)
    doctor_info['specialty'] = doc
    
    ### combine with other docs
    risk_benefit = pd.concat([risk_benefit,doctor_info])
    print(doc)

In [ ]:
risk_benefit.rename(columns={'adj_index':'docid'},inplace=True)

In [ ]:
risk_benefit = risk_benefit.sort_values('docid').reset_index(drop=True)

In [ ]:
risk_benefit.to_csv('results/Risk_Benefit_table.csv',index=False)